https://www.youtube.com/watch?v=uM4u7P2xkO8

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
import pathlib
print(tf.__version__)

2.16.0-rc0


Utilizar los documentos .txt
- Leer los .txt en orden: papel, piedra, tijera
- Meter 700 docs para el entreno
- Y los otros 300 para la prueba

In [ ]:
clases = {"papel" : 0, "piedra" : 1, "tijera" : 2}
num_entrena = 35000
num_prueba = 15000
word_length = 20
palabras_entrena = np.zeros((num_entrena * len(clases), 1, word_length))
clases_entrena = np.zeros((num_entrena * len(clases)))
palabras_prueba = np.zeros((num_prueba * len(clases), 1, word_length))
clases_prueba = np.zeros((num_prueba * len(clases)))

print("Arrays vacíos creados correctamente.")

for i in range(num_entrena):
    for clase in clases:
        palabra = open("dataset_rand\\" + clase + "\\" + str(i) + ".txt", "r").read()
        indice = i + clases[clase] * num_entrena
        for j in range(word_length):
            palabras_entrena[indice][0][j] = int.from_bytes(list(palabra)[j].encode(), byteorder="big")
            clases_entrena[indice] = clases[clase]

print("Array de ENTRENAMIENTO completado con los datos correctamente.")

for i in range(num_entrena, num_prueba + num_entrena):
    for clase in clases:
        palabra = open("dataset_rand\\" + clase + "\\" + str(i) + ".txt", "r").read()
        indice = i + clases[clase] * num_prueba - num_entrena
        for j in range(word_length):
            palabras_prueba[indice][0][j] = int.from_bytes(list(palabra)[j].encode(), byteorder="big")
            clases_prueba[indice] = clases[clase]

print("Array de PRUEBA completado con los datos correctamente.")
print("Proceso completado")
            
# print(palabras_entrena)
# print(clases_entrena)
# print("------------------------------------")
# print(palabras_prueba)
# print(clases_prueba)

Arrays vacíos creados correctamente.
Array de ENTRENAMIENTO completado con los datos correctamente.


In [ ]:
clases = {"papel" : 0, "piedra" : 1, "tijera" : 2}
palabras_entrena = ["sjopgdjtkpapelkamfbr", "qolsjdeltpiedrawlpsr", "wlfncuwptijeraqlmsyp"]
docs_entrena = np.zeros((3, 1, 20))
clases_entrena = np.zeros((3))

for i in range(len(palabras_entrena)):
    for j in range(len(palabras_entrena[i])):
        docs_entrena[i][0][j] = int.from_bytes(list(palabras_entrena[i])[j].encode(), byteorder="big")
        clases_entrena[i] = i
print("docs_entrena")
print(docs_entrena)
print("clases_entrena")
print(clases_entrena)

palabras_prueba = ["holaadiospapelkslpwq", "piedraqjcmalpwjdvmol", "qpalskmcetupfhtijera"]
docs_prueba = np.zeros((3, 1, 20))
clases_prueba = np.zeros((3))

for i in range(len(palabras_prueba)):
    for j in range(len(palabras_prueba[i])):
        docs_prueba[i][0][j] = int.from_bytes(list(palabras_prueba[i])[j].encode(), byteorder="big")
        clases_prueba[i] = i
print("docs_prueba")
print(docs_prueba)
print("clases_prueba")
print(clases_prueba)

Escalar imagenes de [0,255] a [0,1]

In [ ]:
palabras_entrena = palabras_entrena / 126
palabras_prueba = palabras_prueba / 126

Crear la estructura del modelo

In [ ]:
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1,20)), # hace cambios en el formato de la matriz
    tf.keras.layers.Dense(64, activation="relu"), # 64 neuronas, funcion de activación relu
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(3), # 3 neuronas porque hay 3 clases
    tf.keras.layers.Softmax()
])

Compilar modelo

In [ ]:
modelo.compile(optimizer="sgd",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"])

Entrenar modelo

In [ ]:
historial = modelo.fit(palabras_entrena, clases_entrena, epochs=100) #700

Predecir imágenes de prueba

In [ ]:
test_loss, test_acc = modelo.evaluate(palabras_entrena, clases_entrena, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
plt.xlabel("Epochs")
plt.ylabel("Magnitud de pérdida")
plt.plot(historial.history["loss"])

In [ ]:
predicciones = modelo.predict(palabras_prueba)

In [ ]:
for instancia in range(palabras_prueba.shape[0]):
    print(predicciones[instancia])

In [ ]:
# Creación de arreglo para almacenar predicciones
clase_predicha = np.empty(palabras_prueba.shape[0], dtype = "uint8")

for instancia in range(palabras_prueba.shape[0]):
  # almacena clase predicha para una imagen dada
  clase_predicha[instancia] = np.argmax(predicciones[instancia])
  #if clase_predicha[instancia] == clases_prueba[instancia]:
    #print("Probabilidades:", predicciones[instancia],
          #"Clase predicha:", clase_predicha[instancia],
          #"Clase correcta:", clases_prueba[instancia],
          #"La Red Neuronal ACERTÓ")
  #else:
    #print("Probabilidades:", predicciones[instancia],
          #"Clase predicha:", clase_predicha[instancia],
          #"Clase correcta:", clases_prueba[instancia],
          #"La Red Neuronal ERRÓ")
print(clase_predicha)

In [ ]:
matriz = tf.math.confusion_matrix(clases_prueba, clase_predicha)
print("Matriz de Confusión:\n", matriz.numpy())